In [ ]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as so
import math
import multiprocessing as mp
import sys
import time

# === STOP FLAG GLOBALE (per compatibilità Jupyter/Spyder) ===
stop_requested = False

# === PARALLEL ===
def _fit_one(args):
    findex, ar, ff, tvals, ch1vals, ch2vals = args
    def fitfunc(x, A, phi, offset):
        return A * np.cos(2*np.pi*ff*x + phi) + offset
    pp1, _ = so.curve_fit(fitfunc, tvals, ch1vals, p0=[1, 0, 0])
    pp2, _ = so.curve_fit(fitfunc, tvals, ch2vals, p0=[1, 0, 0])
    if pp1[0] < 0:
        pp1[0] *= -1
        pp1[1] += np.pi
    if pp2[0] < 0:
        pp2[0] *= -1
        pp2[1] += np.pi
    A_ratio = pp2[0] / pp1[0]
    phi = (pp2[1] - pp1[1] + np.pi) % (2*np.pi) - np.pi
    return (findex, ar, A_ratio, phi)


if __name__ == "__main__":
    mp.freeze_support()  # utile se un giorno "freezi" lo script

    # ==========================================================================
    #  Parametri dello script
    nper = 2
    npt = 8192
    nf = 200
    f0 = 50        
    f1 = 500e3      
    flag_return = False
    flag_show = True

    # Nuovo flag per disattivare completamente la GUI (più veloce)
    flag_gui = True   # metti False per non aprire alcuna finestra

    fv = np.logspace(np.log10(f0), np.log10(f1), nf) 

    # ==========================================================================
    #  Configurazione base AD2
    ad2 = tdwf.AD2()
    ad2.vdd = 3
    ad2.vss = -3
    ad2.power(True)

    wavegen = tdwf.WaveGen(ad2.hdwf)
    wavegen.w1.ampl = 0.02
    wavegen.w1.func = tdwf.funcSine 
    wavegen.w1.start()
    scope = tdwf.Scope(ad2.hdwf)
    scope.ch1.rng = 5
    scope.ch2.rng = 50
    scope.ch1.avg = True
    scope.ch2.avg = True

    # ==========================================================================
    #   Esecuzione...

    # [1] Crea plot (solo se GUI attiva)
    flag_first = True
    if flag_gui:
        fig, [[ax1, ax3], [ax2, ax4]] = plt.subplots(2,2,figsize=(12,6),
            gridspec_kw={'width_ratios': [1, 2]})
        fig.canvas.manager.set_window_title('Spazzata frequenza')

    # [2] Preparazione dataset
    Am   = np.full((nf, 2), np.nan)
    phim = np.full((nf, 2), np.nan)

    # === PARALLEL ===
    tasks = []  # (findex, ar, ff, tvals, ch1, ch2)

    # === PROGRESS ===
    total_meas = len(fv) * (2 if flag_return else 1)
    done = 0
    t0 = time.time()

    def _progress_line(done, total, extra=""):
        if total <= 0: return
        pct = 100.0 * done / total
        elapsed = time.time() - t0
        rate = done / elapsed if elapsed > 0 else 0.0
        eta = (total - done) / rate if rate > 0 else float('inf')
        eta_s = f"{eta:5.1f}s" if np.isfinite(eta) else "--"
        msg = f"[ACQ] {done:4d}/{total:4d} ({pct:5.1f}%)  ETA {eta_s}  {extra}"
        print("\r" + msg + " " * 10, end="", flush=True)

    # Se la finestra viene chiusa, fermiamo l'acquisizione ma salviamo ciò che c'è
    if flag_gui:
        def on_close(evt):
            global stop_requested
            stop_requested = True
            print("\n[INFO] Finestra chiusa: interrompo le acquisizioni dopo la misura corrente...")
        fig.canvas.mpl_connect('close_event', on_close)

    try:
        # [3] SOLO acquisizione
        for ar in range(2 if flag_return else 1):
            for ii in range(len(fv)):
                if stop_requested:
                    break

                # [3a] indice/frequenza
                if ar == 0:
                    findex = ii
                else:
                    findex = len(fv)-ii-1
                ff = fv[findex]

                # [3b] settaggi sampling
                df = math.ceil(100e6*nper/(npt*ff))
                scope.fs = 100e6/df
                scope.npt = int(scope.fs*nper/ff)
                scope.trig(True, hist=0.01)
                wavegen.w1.freq = ff 

                # [3c] Acquisizione
                scope.sample()

                # Metti in coda per il fit
                tvals  = scope.time.vals.copy()
                ch1val = scope.ch1.vals.copy()
                ch2val = scope.ch2.vals.copy()
                tasks.append((findex, ar, ff, tvals, ch1val, ch2val))

                # GUI: aggiorna solo il tempo-dominio per feedback visivo
                if flag_gui:
                    if flag_first:
                        flag_first = False
                        if flag_show:
                            hp1, = ax1.plot(1000*scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
                            hp2, = ax2.plot(1000*scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
                            ax1.grid(True); ax2.grid(True)
                            ax1.set_xticklabels([])
                            ax1.set_ylabel("Ch1 [V]", fontsize=15)
                            ax2.set_xlabel("Time [msec]", fontsize=15)
                            ax2.set_ylabel("Ch2 [V]", fontsize=15)
                            ax1.set_xlim([0, nper/ff]); ax2.set_xlim([0, nper/ff])
                            ax1.set_title("Starting")

                        # Placeholders Bode (si aggiornano solo a fine fit)
                        hp3A, = ax3.loglog(fv, Am[:, 0], ".", markerfacecolor="none", label="Amp go", color="tab:orange")
                        hp4A, = ax4.semilogx(fv, phim[:, 0], ".", markerfacecolor="none", label="phi go", color="tab:orange")
                        if flag_return:
                            hp3R, = ax3.loglog(fv, Am[:, 1], "v", markerfacecolor="none", label="Amp return", color="tab:blue")
                            hp4R, = ax4.semilogx(fv, phim[:, 1], "v", markerfacecolor="none", label="phi return", color="tab:blue")
                        ax3.grid(True); ax4.grid(True)
                        ax3.set_xticklabels([])
                        ax3.yaxis.tick_right(); ax3.yaxis.set_label_position('right')
                        ax3.set_ylabel("Gain [pure]", fontsize=15)
                        ax4.set_xlabel("Freq [Hz]", fontsize=15)
                        ax4.yaxis.tick_right(); ax4.yaxis.set_label_position('right')
                        ax4.set_yticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi])
                        ax4.set_yticklabels(["-180","-90","0","+90","+180"])
                        ax4.set_ylabel("Phase [deg]", fontsize=15)
                        ax3.legend(); ax4.legend()
                        plt.tight_layout()
                        plt.show(block=False)
                    else:
                        if flag_show:
                            hp1.set_xdata(1000*scope.time.vals)
                            hp1.set_ydata(scope.ch1.vals)
                            hp2.set_xdata(1000*scope.time.vals)
                            hp2.set_ydata(scope.ch2.vals)
                            if ff > 1e3:
                                title = f"{ff/1e3:.1f}kHz"
                            else:
                                title = f"{ff:.1f}Hz"
                            if scope.fs > 1e6:
                                ax1.set_title(title+f" [{scope.npt:d}pts @ {scope.fs/1e6:.1f}MSa/s]")
                            elif scope.fs > 1e3:
                                ax1.set_title(title+f" [{scope.npt:d}pts @ {scope.fs/1e3:.1f}kSa/s]")
                            else:
                                ax1.set_title(title+f" [{scope.npt:d}pts @ {scope.fs:.1f}Sa/s]")
                            ax1.set_xlim([-500*nper/ff, +500*nper/ff])
                            ax2.set_xlim([-500*nper/ff, +500*nper/ff])
                            mi = scope.ch2.vals.min(); ma = scope.ch2.vals.max()
                            dm = ma - mi; m0 = (ma + mi)/2
                            ax2.set_ylim([m0 - 0.6*dm, m0 + 0.6*dm])
                        fig.canvas.draw()
                        fig.canvas.flush_events()
                        plt.pause(0.001)

                # PROGRESS
                done += 1
                _progress_line(done, total_meas, extra=f"f={ff:.1f}Hz")

            if stop_requested:
                break

    except KeyboardInterrupt:
        print("\n[INFO] Interrotto da tastiera (Ctrl+C). Proseguo con i dati raccolti...")

    print("\n[INFO] Avvio l'elaborazione (fit) in parallelo...")

    # === PARALLEL ===
    with mp.Pool(processes=mp.cpu_count()) as pool:
        for findex, ar, A_ratio, phi in pool.imap_unordered(_fit_one, tasks):
            Am[findex, ar] = A_ratio
            phim[findex, ar] = phi

    # Aggiornamento Bode SOLO a fine fit (se GUI attiva)
    if flag_gui and 'hp3A' in locals():
        hp3A.set_ydata(Am[:, 0])
        hp4A.set_ydata(phim[:, 0])
        if flag_return and 'hp3R' in locals():
            hp3R.set_ydata(Am[:, 1])
            hp4R.set_ydata(phim[:, 1])

        ax3.set_xlim([f0, f1])
        ax4.set_xlim([f0, f1])

        if np.any(np.isfinite(Am[:, 0])):
            y_min = 0.5 * np.nanmin(Am[:, 0])
            y_max = 2.0 * np.nanmax(Am[:, 0])
            if np.isfinite(y_min) and np.isfinite(y_max) and (y_max > y_min) and (y_min > 0):
                ax3.set_ylim([y_min, y_max])

        fig.canvas.draw()
        fig.canvas.flush_events()

    # Salvataggio risultati come prima
    data = np.column_stack((fv, Am, phim))
    np.savetxt("AD8031_Gain1e3.txt", data)

    # Chiudi hardware
    ad2.close()

    print("[OK] Completato. Risultati salvati in AD8031_Gain1e3.txt")


Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
[ACQ]  200/ 200 (100.0%)  ETA   0.0s  f=500000.0Hz          
[INFO] Avvio l'elaborazione (fit) in parallelo...
